# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# df = pd.read_csv("../Weather/output_data/cities.csv")
df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
# Configure Gmaps
gmaps.configure(api_key=api)

In [ ]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = df[["Latitude", "Longitude"]].astype(float)

In [ ]:
# Initialize (Heat)map
fig = gmaps.figure()

gmap_layer = gmaps.heatmap_layer(
    locations, 
    weights=df["Humidity"],
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
fig.add_layer(gmap_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#     - A max temperature lower than 80 degrees but higher than 70.
#     - Wind speed less than 10 mph.
#     - Zero cloudiness.
#     - Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by 
#       your API requests to a reasonable number.

In [ ]:
vacation_spot = df[
    (df['Max Temp(f)'] > 70) & 
    (df['Max Temp(f)'] < 80) &
    (df['Wind Speed'] < 10.00) &
    (df['Cloudiness'] > 0) &
    (df['Cloudiness'] < 5)
]
vacation_spot


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store into hotel_df
hotel_df = pd.DataFrame(data=vacation_spot)


In [ ]:
# Add a "Hotel Name" column into the DataFrame
hotel_df.loc[:,'Hotel Name'] = np.nan

In [ ]:
# Set parameters to search for hotels within 5000 meters
target_radius = 5000
target_type = "hotel"
target_search = "hotel"


In [ ]:
# Hit the Google Places API for each city's coordinates

hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Latitude"].astype(str) +"," + hotel_df.loc[:,"Longitude"].astype(str)

hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": api
    }

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5km of {location}.")

hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
new_fig = fig = gmaps.figure()

markers = gmaps.marker_layer(locations, hotel_info)
new_fig.add_layer(gmap_layer)
new_fig.add_layer(markers)
new_fig


# Display figure
